# Dgemm model generation

In this notebook, we take an existing dgemm model from an old zip archive and modify it to create new models (with higher and lower variability).

In [1]:
!unzip -p factor_exp/2/grenoble_2020-01-16_1909229.zip dgemm_128.yaml > /tmp/dgemm.yaml
!sha256sum /tmp/dgemm.yaml

78820b4f0219eadb81dfeb33f40f4714c2b3b64c7ee6f7ef9a74b77b138cd039  /tmp/dgemm.yaml


In [2]:
import yaml
import copy
import datetime

with open('/tmp/dgemm.yaml') as f:
    model = yaml.load(f, Loader=yaml.SafeLoader)
model['info']

{'cluster': 'dahu',
 'experiment_date': '2020-01-14 09:08:32',
 'expfile_hash': '84746a24489a46f4cf7b4c6accf8d5540c9001dc1d4ab054622612039adb8d68',
 'function': 'dgemm',
 'jobid': 1908315}

In [3]:
model['metadata']

{'archive_file': '../mpi_calibration/dahu/blas/block_size/6/grenoble_2020-01-14_1908315.zip',
 'cashew_git': 'ca8a64e6735b60bc60c49342320befcdeaf55f90',
 'file_creation_date': '2020-01-16 17:03:53.485894'}

In [4]:
model['model'][:2]

[{'avg_gflops': 27.791429198588983,
  'cpu': 0,
  'cpu_id': 2,
  'intercept': 2.3131041666666666e-06,
  'intercept_residual': 2.502215256002448e-07,
  'mk': 1.548172812203589e-09,
  'mk_residual': 2.954739221845529e-11,
  'mn': -2.607011073797494e-11,
  'mn_residual': -1.7276391852417193e-11,
  'mnk': 7.109456643397961e-11,
  'mnk_residual': 3.094747443316429e-13,
  'nk': 4.708826525721923e-09,
  'nk_residual': -7.705694556409434e-11,
  'node': 1,
  'tvalue_mk': 2.2326711115763165,
  'tvalue_mk_residual': 0.1861172248510929,
  'tvalue_mn': -0.30938859812470476,
  'tvalue_mn_residual': -0.8955226383620872,
  'tvalue_mnk': 91.04221458652901,
  'tvalue_mnk_residual': 1.7309866186048979,
  'tvalue_nk': 6.790754152593327,
  'tvalue_nk_residual': -0.48537700917420395},
 {'avg_gflops': 29.089353709306522,
  'cpu': 1,
  'cpu_id': 3,
  'intercept': 2.2221875e-06,
  'intercept_residual': 4.6807654175550695e-07,
  'mk': 1.493565993389137e-09,
  'mk_residual': 9.174467832725285e-11,
  'mn': -2.428

In [5]:
def apply_factor_cpu(cpu_model, factor):
    for key, value in cpu_model.items():
        if key.endswith('_residual'):
            cpu_model[key] *= factor

def apply_factor(model, factor):
    for cpu_model in model['model']:
        apply_factor_cpu(cpu_model, factor)
    model['metadata']['comment'] = 'Multiplied all the residual parameters by a factor %e' % factor
    model['metadata']['file_creation_date'] = str(datetime.datetime.now())

In [6]:
for factor in [0.1, 1, 10]:
    tmp = copy.deepcopy(model)
    apply_factor(tmp, factor)
    cpu_model = tmp['model'][4]  # taking some arbitrary CPU
    print('mnk_residual for CPU %d with factor %e: %e' % (cpu_model['cpu_id'], factor, cpu_model['mnk_residual']))
    with open('/tmp/dgemm_sigma_%.0e.yaml' % factor, 'w') as f:
        f.write(yaml.dump(tmp))

mnk_residual for CPU 6 with factor 1.000000e-01: 3.446646e-14
mnk_residual for CPU 6 with factor 1.000000e+00: 3.446646e-13
mnk_residual for CPU 6 with factor 1.000000e+01: 3.446646e-12


In [7]:
!sha256sum /tmp/dgemm_sigma_*

6bd1ed9bb9e9bc4c5919a385b5d1cd135b79c9271088621f0bb55f2606ddc88a  /tmp/dgemm_sigma_1e+00.yaml
1fe5f5f8e8d1b0a1224c2e365bfa1829cd11ff1fb21fb5d5f9e9902df7b719e0  /tmp/dgemm_sigma_1e+01.yaml
7be21f36a46bfe981f7d36b14026f62795f3a8a8714f95047b3d62a4cb6bbaa1  /tmp/dgemm_sigma_1e-01.yaml
